<a href="https://colab.research.google.com/github/mhenson1989/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop2.tgz
!tar xf spark-3.3.0-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [912 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-01 02:26:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.95MB/s    in 0.2s    

2022-09-01 02:26:17 (4.95 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-VineReview-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [10]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [13]:
# step 1 - filter and create df where total_votes is equal to or greater than 20
vote_count_df = vine_df.filter("total_votes >= 20")
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [14]:
# # Step 2 - filter vote_count_df to for helpful_votes/total_votes >= 50%
fifty_df = vote_count_df.filter(vote_count_df["helpful_votes"]/vote_count_df["total_votes"] >= 0.5)
fifty_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [15]:
# Step 3 - filter above_50_df that retrieves all rows where vine =='Y'
paid_vine_df = fifty_df.filter("vine == 'Y'")
paid_vine_df.show()
#analysis is indicating that no ratings with more than twenty votes where the helpful votes by total votes greater than 50% was a paid Vine review.

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [17]:
# Step 4 - filter above_50_df that retrieves all rows where vine =='N'
unpaid_vine_df = fifty_df.filter("vine == 'N'")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

In [23]:
# Step 5a - Determine Total Number of Reviews
total_review_count = fifty_df.count()
total_review_count

403807

In [24]:
# Step 5b - Total Number of 5-Star Reviews
total_five_count = fifty_df.filter("star_rating == 5").count()
total_five_count


242889

In [33]:
#Step 5c - Total Unpaid 5-Star Reviews
unpaid_five_df = unpaid_vine_df.filter("star_rating == 5").count()
unpaid_five_df

242889

In [34]:
#Step 5d - Total Paid 5-Star Reviews
paid_five_df = paid_vine_df.filter("star_rating == 5").count()
paid_five_df

0

In [37]:
# Step 5e - % of 5 Star Reviews (paid)
paid_percent_df = (paid_five_df/total_five_count)*100

paid_percent_df

0.0

In [38]:
# Step 5f - % of 5 Star Reviews (unpaid)
unpaid_percent_df = (unpaid_five_df/total_five_count)*100

unpaid_percent_df


100.0